#📌 Extracción

In [ ]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.1 MB 2.4 MB/s eta 0:00:04
   ------ --------------------------------- 1.3/8.1 MB 2.7 MB/s eta 0:00:03
   ---------- ----------------------------- 2.1/8.1 MB 2.7 MB/s eta 0:00:03
   --------------- ------------------------ 3.1/8.1 MB 3.2 MB/s eta 0:00:02
   -------------------- ------------------- 4.2/8.1 MB 3.4 MB/s eta 0:00:02
   ------------------------ --------------- 5.0/8.1 MB 3.5 MB/s eta 0:00:01
   ---------------------------- ----------- 5.8/8.1 MB 3.2 MB/s eta 0:00:01
   ------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.0 MB 5.4 MB/s eta 0:00:02
   ----- ---------------------------------- 1.0/8.0 MB 3.4 MB/s eta 0:00:03
   ----------- ---------------------------- 2.4/8.0 MB 4.5 MB/s eta 0:00:02
   ------------------- -------------------- 3.9/8.0 MB 5.1 MB/s eta 0:00:01
   -------------------------- ------------- 5.2/8.0 MB 5.5 MB/s eta 0:00:01
   ------------------------------- -------- 6.3/8.0 MB 5.3 MB/s eta 0:00:01
   ------------------------------------ --- 7.3/8.0 MB 5.2 MB/s eta 0:00:01
   -------------------------------------- - 7.6/8.0 MB 4.7 MB/s eta 0:00:01
   ---------------------------------------  7.9/8.0 MB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 4.3 MB/s  0:00:01
   ---------------

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json 
with open("TelecomX_Data.json", "r") as f:
    data = json.load(f)

#🔧 Transformación

In [5]:
pd.json_normalize(data)

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,9987-LUTYD,No,Female,0,No,No,13,Yes,No,DSL,...,No,No,Yes,No,No,One year,No,Mailed check,55.15,742.9
7263,9992-RRAMN,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber optic,...,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.7
7264,9992-UJOEL,No,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75
7265,9993-LHIEB,No,Male,0,Yes,Yes,67,Yes,No,DSL,...,No,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65


#📊 Carga y análisis

#📄Informe final